In [1]:
from gensim.corpora import MmCorpus, Dictionary
from gensim import similarities
import numpy as np
from tables import *
import time
import multiprocessing as mp
import csv

In [2]:
tfidf_path = "/mnt/disks/sdb/data/pub_full_tfidf.mm"
ids_path = "/mnt/disks/sdb/data/abs_cits.txt"
path = "/mnt/disks/sdc/data_journals/"
out_path = path + "sim/"
dict_path = "/mnt/disks/sdb/data/dct.p"
info_path = "/mnt/disks/sdb/data/pub_exp_info.txt"

In [3]:
dictionary = Dictionary.load(dict_path)
num_features= len(dictionary)
del dictionary
mm_tfidf = MmCorpus(tfidf_path)
with open(ids_path,'r') as f:
    abs_cits = f.readlines()


In [4]:
with open(ids_path,'r') as f:
    abs_cits = f.readlines()
abs_cits = np.array([int(x.strip()) for x in abs_cits])
abs_dict = {x:i for i,x in enumerate(abs_cits)}
del abs_cits

In [5]:
import pickle
with open(path+'dict_journals_year.p','rb') as f:
    dict_journals_years = pickle.load(f)
journals_years = list(dict_journals_years.keys())
nb_journals_years = len(journals_years)

In [15]:
batch_size = 1000

In [7]:
def compute_sim(inqueue,id_out):
    for journal_year in iter(inqueue.get,sentinel):
        journal = journal_year[0]
        list_arts = dict_journals_years[journal_year]
        comps = []
        for art in list_arts:
            if art in abs_dict.keys():
                comps.append(art)
        del list_arts
                      
        nb_IDs = len(comps) 
        nb_batches = int(np.ceil(nb_IDs/batch_size))
        if nb_batches == 1:
            tfidf_b = [mm_tfidf[abs_dict[k]] for k in comps]
            index = similarities.SparseMatrixSimilarity(tfidf_b,num_features=num_features)
            sim = index[tfidf_b]
            comp_map_b = {i:x for i,x in enumerate(comps)}
            pairs_id = np.tril_indices(sim.shape[0],-1)
            pairs_sim = []
            for j in range(len(pairs_id[0])):
                pairs_sim.append([comp_map_b[pairs_id[0][j]],comp_map_b[pairs_id[1][j]],sim[pairs_id[0][j],pairs_id[1][j]]])
            handle_output([pairs_sim,journal,True,id_out])
       
        elif nb_batches > 1:
            for i_b in range(nb_batches-1):
                s_i = i_b*batch_size
                if i_b == nb_batches-2:
                    e_i = nb_IDs
                else:
                    e_i = (i_b+1)*batch_size
                comps_i = comps[s_i:e_i]
                tfidf_i = [mm_tfidf[abs_dict[k]] for k in comps_i]
                index = similarities.SparseMatrixSimilarity(tfidf_i,num_features=num_features)
                comp_map_i = {i:x for i,x in enumerate(comps_i)}
                for j_b in range(i_b,nb_batches-1):
                    s_j = j_b*batch_size
                    if j_b == nb_batches-2:
                        e_j = nb_IDs
                    else:
                        e_j = (j_b+1)*batch_size
                    comps_j = comps[s_j:e_j]
                    tfidf_j = [mm_tfidf[abs_dict[k]] for k in comps_j]
                    sim = index[tfidf_j]
                    comp_map_j = {i:x for i,x in enumerate(comps_j)}
                    if i_b == j_b:
                        pairs_id = np.tril_indices(sim.shape[0],-1)
                        pairs_sim = []
                        for j in range(len(pairs_id[0])):
                            pairs_sim.append([comp_map_i[pairs_id[0][j]],comp_map_j[pairs_id[1][j]],sim[pairs_id[0][j],pairs_id[1][j]]])
                    else:
                        dim_mat = sim.shape
                        pairs_sim = []
                        for dim_i in range(dim_mat[0]):
                            for dim_j in range(dim_mat[1]):
                                pairs_sim.append([comp_map_i[dim_j],comp_map_j[dim_i],sim[dim_i,dim_j]])
                    if i_b == nb_batches-2 and j_b == nb_batches-2:
                        handle_output([pairs_sim,journal,True,id_out])
                    else:
                        handle_output([pairs_sim,journal,False,id_out])
        else:
            handle_output([[],None,True,id_out])


### Out to txt

In [12]:
def handle_output(args):
    if len(args[0]) > 0:
        cont = False
        while True:
            try:
                fo = open('{}output_{}.txt'.format(out_path,args[3]),'a')
                for line in args[0]: 
                    fo.write('{0},{1},{2:.4f}\n'.format(line[0],line[1],line[2]))
                fo.flush()
                fo.close()
                break
            except IOError:
                time.sleep(0.5)

    if args[2]:
        elapsed_time = time.time() - start_time
        elapsed_time_h = np.round(elapsed_time/3600,2)
        while True:
            try:
#                 fp = open('{}../progress.txt'.format(out_path),'r')
#                 counter = 1
#                 for counter, l in enumerate(fp):
#                     pass
#                 fp.close()
                
                fp = open('{}../progress.txt'.format(out_path),'a')
                #fp.write('Progress: {}/{} journals, Time since start: {}\n'.format(counter+2,nb_journals_years,elapsed_time_h))
                fp.write('Progress: {}/{} journals, Time since start: {}\n'.format('',nb_journals_years,elapsed_time_h))    
                fp.flush()
                fp.close()
                break
            except IOError:
                time.sleep(0.2)
         




In [9]:
import shutil
import os

In [16]:
shutil.rmtree(out_path)

In [17]:
if not os.path.exists(out_path):
    os.makedirs(out_path)

start_time = time.time()
fp = open('{}../progress.txt'.format(out_path),'w')
fp.close()


num_processes = mp.cpu_count()-1
sentinel = None
manager = mp.Manager()
inqueue = mp.Queue()
jobs = []


for i in range(num_processes):
    p = mp.Process(target=compute_sim, args=(inqueue,i,))
    jobs.append(p)
    p.start()
idx_journals = np.arange(nb_journals_years)
np.random.shuffle(idx_journals)
for i in idx_journals:
    inqueue.put(journals_years[i])
for i in range(num_processes):
    # Send the sentinal to tell Simulation to end
    inqueue.put(sentinel)
for p in jobs:
    p.join()

proc.join()

NameError: name 'proc' is not defined

if not os.path.exists(out_path):
    os.makedirs(out_path)

start_time = time.time()
fp = open('{}../progress.txt'.format(out_path),'w')
fp.close()


nb_outputs = 7
num_processes = mp.cpu_count()-nb_outputs-1
sentinel = None
manager = mp.Manager()
output = mp.Queue()
inqueue = mp.Queue()
jobs = []

for id_out in range(nb_outputs):
    proc = mp.Process(target=handle_output, args=(output,))
    proc.start()

for i in range(num_processes):
    p = mp.Process(target=compute_sim, args=(inqueue, output))
    jobs.append(p)
    p.start()
idx_journals = np.arange(nb_journals)
np.random.shuffle(idx_journals)
for i in idx_journals:
    inqueue.put(journals[i])
for i in range(num_processes):
    # Send the sentinal to tell Simulation to end
    inqueue.put(sentinel)
for p in jobs:
    p.join()
output.put(None)
proc.join()

In [ ]:
queue_cont = []
while True:
    args = output.get()
    if args:
        queue_cont.append(args)
    else:
        break
        

In [ ]:
len(queue_cont)

In [ ]:
tfidf_i = [tfidf[0]] 
index = similarities.SparseMatrixSimilarity(tfidf_i,num_features=num_features)
tfidf_j = [tfidf[1]]


In [ ]:
tfidf_i

In [ ]:
tfidf_j

In [ ]:
sim = index[tfidf_j]

sim

In [ ]:
list(range(nb_batches-1))

In [ ]:
len(tfidf_i)

In [ ]:
from matplotlib import pyplot as plt
plt.pcolor(temp)
plt.show()

In [ ]:
total = 0
for i in output:
    total += len(i[0])
total

In [ ]:
len(np.tril_indices(nb_IDs,-1)[0])

In [ ]:
nb_IDs*(nb_IDs-1)/2

In [ ]:
temp = list(abs_dict.keys())

In [ ]:
len(comps)

In [ ]:
temp = []
for i_b in range(nb_batches):#range(nb_batch):
    for j_b in range(i_b,nb_batches):
        temp.append([i_b,j_b])

In [ ]:
temp